In [217]:
import pandas as pd

data = pd.read_excel("Nov.xlsx").sort_values(by=["Ext Orderno"]).reset_index().drop(columns=['index'])
data["Ext Orderno"] = data["Ext Orderno"].astype(int)
data["Skucode"] = data["Skucode"].astype(int)
data["totalQty"] = data["totalQty"].astype(int)

product = data[["Skucode", "SkuName"]].drop_duplicates().reset_index().drop(columns=['index'])

order = data[["Ext Orderno", "Skucode", "totalQty"]]


In [218]:
#Aggregate the Order Number by sku_qty so that each order only takes a row

order_meged = pd.DataFrame(columns=['Ext Orderno', 'sku_qty', 'Status'])


tem_orderno = 0
for index, row in order.iterrows():
    if row['Ext Orderno'] != tem_orderno:
        tem_orderno = row['Ext Orderno']
        sku_qty = {row['Skucode']: row['totalQty']}
        order_meged = order_meged.append({'Ext Orderno': row['Ext Orderno'], 'sku_qty': sku_qty}, ignore_index=True)
    elif row['Ext Orderno'] == tem_orderno:
        if row['Skucode'] in sku_qty:        
            sku_qty.update({row['Skucode']: sku_qty.get(row['Skucode'])+row['totalQty']})
        else: 
            sku_qty.update({row['Skucode']: row['totalQty']})

order_meged['Status'] = False
order = order_meged
order.head(5)

/var/folders/rx/s42f27kn5j5ddqzlhz7qr2yw0000gn/T/ipykernel_14533/442861628.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  order_meged = order_meged.append({'Ext Orderno': row['Ext Orderno'], 'sku_qty': sku_qty}, ignore_index=True)
/var/folders/rx/s42f27kn5j5ddqzlhz7qr2yw0000gn/T/ipykernel_14533/442861628.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  order_meged = order_meged.append({'Ext Orderno': row['Ext Orderno'], 'sku_qty': sku_qty}, ignore_index=True)
/var/folders/rx/s42f27kn5j5ddqzlhz7qr2yw0000gn/T/ipykernel_14533/442861628.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  order_meged = order_meged.append({'Ext Orderno': row['Ext Orderno'], 'sku_qty': sku_qty}, ignore_index=True)
/var/folders/rx/s42f27

,Ext Orderno,sku_qty,Status
0,2102702909,"{743195: 1, 236216: 2}",False
1,2102706570,"{161034: 16, 137166: 6}",False
2,2102714203,"{416537: 1, 513861: 1, 892687: 2, 968271: 1, 9...",False
3,2102716313,"{936906: 10, 778407: 2, 129809: 1}",False
4,2102716315,"{936906: 10, 703538: 2}",False


In [219]:
#order['totalQty'] = 0
order_qty = []

for index, row in order.iterrows():
    qty = 0
    for item in row['sku_qty']:
        #r_qty = int(row['sku_qty'][item])
        r_qty = row['sku_qty'][item]
        qty += r_qty
    order_qty.append(qty)

order_totalQty = pd.DataFrame(order_qty)
order_totalQty = order_totalQty.rename(columns={0: 'totalQty'})
order['totalQty'] = order_totalQty


In [220]:
#Show the Stock List for preparing the stock

stock_cal = data[["Skucode", "totalQty"]].sort_values(by=["Skucode"]).reset_index().drop(columns="index")
stock = stock_cal.groupby("Skucode")["totalQty"].sum().reset_index()

In [221]:
#Create two pd dataframe as the rack simulation 

#rack_A = pd.DataFrame(index=range(100), columns=["Ext Orderno", "totalQty"])
#rack_B = pd.DataFrame(index=range(100), columns=["Ext Orderno", "totalQty"])
rack_All = pd.DataFrame(index=range(500), columns=["Ext Orderno", "totalQty"])
#rack_A["totalQty"] = rack_B["totalQty"] = 
rack_All["totalQty"] = 0

#[data]: Ext Orderno, Skucode, SkuNme, totalQty
#[product]: SKucode, SkuName
#[order]: Ext Orderno, sku_qty, Status, totalQty
#[stock]: Skucode, totalQty
#[rack_A][rack_B]: Ext Orderno, totalQty

In [222]:
#Build Correlation Matrix

size = len(product)
corr = [[0 for x in range(size)] for y in range(size)] 

#for index, row in order.iterrows():
#product['Skucode'][0]

#Get Index from a value -> For Correlation Matrix
def get_index(df, col, value):
    return int(df[df[col]==value].index[0])
#User Guide
#print(get_index(product, 'Skucode', 993097))

product.tail(5)




,Skucode,SkuName
1091,902973,Redoxon維多C 3重功效軟糖 維他命C+D+鋅 100粒
1092,180752,GNC兒童DHA軟糖 120粒
1093,69674,Dettol滴露 x Collar酒精搓手液 經典松木(3+1支優惠裝) 50毫升
1094,235358,Mannings萬寧抗菌松木濕巾 10片 x 3包
1095,991661,GNC酵素+益生菌250億 (淨腸配方) 30粒


In [223]:
print(size)

1096


In [224]:
#By Quantity
'''for idx, row in order.iterrows():
    for item_A in row["sku_qty"]:
        for item_B in  row["sku_qty"]:
            if item_A != item_B:
                corr[get_index(product, 'Skucode', item_A)][get_index(product, 'Skucode', item_B)] += row["sku_qty"][item_B]/2
            elif len(row["sku_qty"]) == 1:
                corr[get_index(product, 'Skucode', item_A)][get_index(product, 'Skucode', item_B)] += row["sku_qty"][item_B]'''

#All treat them as 1
for idx, row in order.iterrows():
    for item_A in row["sku_qty"]:
        for item_B in  row["sku_qty"]:
            if item_A != item_B:
                corr[get_index(product, 'Skucode', item_A)][get_index(product, 'Skucode', item_B)] += 1

In [225]:
max = 0
row_max = 0
col_max = 0
depot = 0

for row in range(size):
    for col in range(size):
        if row != col: 
            corr[col][row] += corr[row][col]
            corr[row][col] = corr[col][row]
        if corr[col][row] > max:
            max = corr[col][row]
            row_max = row
            col_max = col

if stock['totalQty'][row_max] > stock['totalQty'][col_max]:
    depot = row_max
else: 
    depot = col_max

max += 1

print(max, row_max, col_max)

for row in range(size):
    for col in range(size):
        corr[row][col] = max - corr[row][col]


109 135 69


In [226]:
#df = pd.DataFrame(corr)
stock.sample(5)


,Skucode,totalQty
238,218198,1
882,824284,2
891,830000,1
333,315432,1
472,462804,1


In [227]:
'''def normalize(dataset):
    dataNorm=((dataset-dataset.min())/(dataset.max()-dataset.min()))
    return dataNorm'''

'def normalize(dataset):\n    dataNorm=((dataset-dataset.min())/(dataset.max()-dataset.min()))\n    return dataNorm'

In [228]:
'''from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

#ortool testing
"""Simple Vehicles Routing Problem (VRP).

   This is a sample using the routing library python wrapper to solve a VRP
   problem.
   A description of the problem can be found here:
   http://en.wikipedia.org/wiki/Vehicle_routing_problem.

   Distances are in meters.
"""

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = corr
    data['num_vehicles'] = 1
    data['depot'] = depot
    return data

def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    max_route_distance = 0
    routingSol = []
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0

        routeSol = []
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            routeSol.append(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        routeSol.append(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        print(plan_output)
        routingSol.append(routeSol)
        #routingSol.append(plan_output)
        #max_route_distance = max(route_distance, max_route_distance)
    #print('Maximum of the route distances: {}m'.format(max_route_distance))
    return routingSol

def get_routes(solution, routing, manager):
  """Get vehicle routes from a solution and store them in an array."""
  # Get vehicle routes and store them in a two dimensional array whose
  # i,j entry is the jth location visited by vehicle i along its route.
  routes = []
  for route_nbr in range(routing.vehicles()):
    index = routing.Start(route_nbr)
    route = [manager.IndexToNode(index)]
    while not routing.IsEnd(index):
      index = solution.Value(routing.NextVar(index))
      route.append(manager.IndexToNode(index))
    routes.append(route)
  return routes



def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()
    result = []

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        300000000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        result = print_solution(data, manager, routing, solution)
    else:
        print('No solution found !')
    
    return result


if __name__ == '__main__':
    resultingSol = main()
'''

'from ortools.constraint_solver import routing_enums_pb2\nfrom ortools.constraint_solver import pywrapcp\n\n#ortool testing\n"""Simple Vehicles Routing Problem (VRP).\n\n   This is a sample using the routing library python wrapper to solve a VRP\n   problem.\n   A description of the problem can be found here:\n   http://en.wikipedia.org/wiki/Vehicle_routing_problem.\n\n   Distances are in meters.\n"""\n\ndef create_data_model():\n    """Stores the data for the problem."""\n    data = {}\n    data[\'distance_matrix\'] = corr\n    data[\'num_vehicles\'] = 1\n    data[\'depot\'] = depot\n    return data\n\ndef print_solution(data, manager, routing, solution):\n    """Prints solution on console."""\n    print(f\'Objective: {solution.ObjectiveValue()}\')\n    max_route_distance = 0\n    routingSol = []\n    for vehicle_id in range(data[\'num_vehicles\']):\n        index = routing.Start(vehicle_id)\n        plan_output = \'Route for vehicle {}:\n\'.format(vehicle_id)\n        route_distance 

In [229]:
#Initialise and function

def find_order_no(skucode: int):
    if type(skucode) == str: skucode = int(skucode)
    #Enter stock no, return the order no that contain(s) the stock
    order_no = []
    for index, row in order.iterrows():
        #print(row['sku_qty'])
        if row['Status'] == True:
            continue
        elif skucode in row['sku_qty']:
            order_no.append(row['Ext Orderno'])
    return order_no

def check_complete(orderno):
    #fulfil: 1, not fulfil: -1
    rack_qty = rack_All.loc[rack_All['Ext Orderno'] == orderno]['totalQty'].iloc[0]
    order_qty = order.loc[order['Ext Orderno'] == int(orderno)]['totalQty'].iloc[0]
    if rack_qty == order_qty:
        return 1
    else: return -1

def get_next_rack():
    return rack_All.loc[rack_All['Ext Orderno'] == '']['Ext Orderno'].index[0]

def get_rack_no(orderno):
    return rack_All[rack_All['Ext Orderno'] == orderno]['Ext Orderno'].index[0]

def get_order_no(orderno):
    return order[order['Ext Orderno'] == orderno]['Ext Orderno'].index[0]

'''def place_order(stockno: int):
    skucode = get_skucode_from_stockno(stockno)
    orders = find_order_no(skucode) 
    
    for item in orders:
        print("main loop:", item)
        if order[order['Ext Orderno'] == item]['Status'].iloc[0] == False:
            if check_on_rack(item) == 1:
                #new rack
                rack_no = get_rack_no(item)
                print(1.1)
                rack_All['totalQty'][rack_no] += order[order['Ext Orderno']==item]['sku_qty'].iloc[0].get(product['Skucode'][stockno])
                print(1.2, rack_All['totalQty'][rack_no], order['totalQty'][order[order['Ext Orderno']==item]['totalQty'].index[0]])
                order['totalQty'][order[order['Ext Orderno']==item]['totalQty'].index[0]] -= rack_All['totalQty'][rack_All[rack_All['Ext Orderno']==item]['totalQty'].index[0]]
                print(1)
            elif check_on_rack(item) == -1:
                #already exist rack
                new_rack_no = get_next_rack()
                print(2.1)
                rack_All['Ext Orderno'][new_rack_no] = item
                print(2.2)
                rack_All['totalQty'][new_rack_no] += order[order['Ext Orderno']==item]['sku_qty'].iloc[0].get(product['Skucode'][stockno])
                print(2.3)
                order['totalQty'][order[order['Ext Orderno']==item]['totalQty'].index[0]] -= rack_All['totalQty'][rack_All[rack_All['Ext Orderno']==item]['totalQty'].index[0]]
                print(2)
            #check complete
            if check_complete(item) == 1:
                print(3.3)
                print("==",stock_pick_order[0])
                stock_pick_order.remove(stockno)
                print("++",stock_pick_order[0])
                order['Status'][order[order['Ext Orderno']==item]['totalQty'].index[0]] = True
                rack_All['totalQty'][rack_All[rack_All['Ext Orderno']==item]['totalQty'].index[0]] = 0
                rack_All['Ext Orderno'][rack_All[rack_All['Ext Orderno']==item]['totalQty'].index[0]] = ''
                print(3)
                return 1
    return -1
'''
def check_on_rack(orderno):
    #int
    if int(orderno) in rack_All['Ext Orderno'].values:
        return 1
    else: return -1

def get_skucode_from_stockno(stockno):
    return product['Skucode'][stockno]

def get_stock_no(skucode):
    return stock[stock['Skucode']==skucode]['totalQty'].iloc[0]

def clear_rack(rack_row_no):
    rack_All['totalQty'][rack_row_no] = 0
    rack_All['Ext Orderno'][rack_row_no] = ''

def mark_order_status(order_row_no):
    order['Status'][order_row_no] = True

def enough_rack(orderno):
    #return 1 -1 
    #calculate empty rack
    empty_rack = rack_All[rack_All['Ext Orderno'] == ''].count().iloc[0]
    sku_on_rack_no = order[order['Ext Orderno'] == orderno].count().iloc[0]

    rack_req = 0

    for index, row in order.iterrows():
        #print(row['sku_qty'])
        if orderno in row['sku_qty']:
            rack_req+= 1

    if rack_req <= empty_rack + sku_on_rack_no:
    #if  empty_rack > 0:
        return 1
    else: return 0




In [230]:
import random

#Random List
stock_pick_order_copy = list(range(len(product)))
random.shuffle(stock_pick_order_copy)
stock_pick_order_copy

[298,
 760,
 826,
 102,
 69,
 393,
 1075,
 497,
 1017,
 991,
 949,
 842,
 83,
 661,
 344,
 918,
 195,
 738,
 993,
 89,
 619,
 664,
 385,
 382,
 863,
 460,
 958,
 187,
 595,
 683,
 1041,
 889,
 373,
 589,
 860,
 714,
 378,
 1028,
 732,
 469,
 1015,
 802,
 782,
 641,
 309,
 7,
 61,
 137,
 709,
 548,
 857,
 263,
 201,
 319,
 740,
 196,
 977,
 866,
 521,
 1002,
 915,
 516,
 849,
 583,
 518,
 329,
 796,
 803,
 870,
 967,
 75,
 537,
 568,
 591,
 625,
 158,
 786,
 135,
 1043,
 1095,
 49,
 480,
 119,
 980,
 635,
 824,
 677,
 56,
 579,
 233,
 282,
 888,
 467,
 288,
 1049,
 1055,
 253,
 937,
 199,
 1029,
 156,
 380,
 326,
 472,
 488,
 645,
 367,
 82,
 264,
 837,
 383,
 586,
 404,
 468,
 976,
 512,
 361,
 259,
 349,
 767,
 491,
 166,
 948,
 219,
 538,
 154,
 108,
 897,
 203,
 470,
 396,
 260,
 592,
 352,
 513,
 885,
 1091,
 178,
 278,
 1085,
 735,
 360,
 794,
 455,
 345,
 223,
 355,
 413,
 577,
 828,
 145,
 605,
 63,
 722,
 1078,
 551,
 336,
 879,
 431,
 275,
 218,
 657,
 293,
 539,
 34,
 810,
 2

In [231]:
#===================== Test Start Under this line =====================
#stock_pick_order_copy = resultingSol[0]

rack_All['Ext Orderno'] = ''
rack_All['totalQty'] = 0

stock_pick_order = stock_pick_order_copy
stock_pick_order

[298,
 760,
 826,
 102,
 69,
 393,
 1075,
 497,
 1017,
 991,
 949,
 842,
 83,
 661,
 344,
 918,
 195,
 738,
 993,
 89,
 619,
 664,
 385,
 382,
 863,
 460,
 958,
 187,
 595,
 683,
 1041,
 889,
 373,
 589,
 860,
 714,
 378,
 1028,
 732,
 469,
 1015,
 802,
 782,
 641,
 309,
 7,
 61,
 137,
 709,
 548,
 857,
 263,
 201,
 319,
 740,
 196,
 977,
 866,
 521,
 1002,
 915,
 516,
 849,
 583,
 518,
 329,
 796,
 803,
 870,
 967,
 75,
 537,
 568,
 591,
 625,
 158,
 786,
 135,
 1043,
 1095,
 49,
 480,
 119,
 980,
 635,
 824,
 677,
 56,
 579,
 233,
 282,
 888,
 467,
 288,
 1049,
 1055,
 253,
 937,
 199,
 1029,
 156,
 380,
 326,
 472,
 488,
 645,
 367,
 82,
 264,
 837,
 383,
 586,
 404,
 468,
 976,
 512,
 361,
 259,
 349,
 767,
 491,
 166,
 948,
 219,
 538,
 154,
 108,
 897,
 203,
 470,
 396,
 260,
 592,
 352,
 513,
 885,
 1091,
 178,
 278,
 1085,
 735,
 360,
 794,
 455,
 345,
 223,
 355,
 413,
 577,
 828,
 145,
 605,
 63,
 722,
 1078,
 551,
 336,
 879,
 431,
 275,
 218,
 657,
 293,
 539,
 34,
 810,
 2

In [232]:
counter = 0
locus = [] 
previous = 0

while len(stock_pick_order) > 0:
    if previous == stock_pick_order[counter]:
        stock_pick_order.remove(stock_pick_order[counter])
    #print(len(stock_pick_order), stock_pick_order[counter])
    stock_pick_no = stock_pick_order[counter]
    skucode = get_skucode_from_stockno(stock_pick_no)
    if enough_rack(skucode) == -1:
        if len(stock_pick_order) >= counter:
            counter += 1
        else: counter = 0
    elif enough_rack(skucode) == 1:
        stock_pick_order[counter]
        ordernos = find_order_no(skucode)
        #check how many of these order numbers are on the rack
        allocated_stock = 0
        for orderno in ordernos:
            #if orders are on the rack
            if check_on_rack(orderno) == 1:
                rack_row_no = get_rack_no(orderno)
            elif check_on_rack(orderno) == -1:
                rack_row_no = get_next_rack()
                rack_All['Ext Orderno'][rack_row_no] = orderno
            #print(rack_row_no)
            locus.append(rack_row_no)
            #print(len(locus))
            #get order row number
            order_row_no = get_order_no(orderno)
            #add corresponding order quantity to rackall
            rack_All['totalQty'][rack_row_no] += order['sku_qty'][order_row_no].get(skucode)
            allocated_stock += order['sku_qty'][order_row_no].get(skucode)
            #check if the rack is completed or not
            if check_complete(orderno) == 1:
                clear_rack(rack_row_no)
                mark_order_status(order_row_no)
        
        stock_total = get_stock_no(skucode)
        if stock_total == allocated_stock:
            stock_pick_order.remove(stock_pick_no)
            counter = 0
        previous = stock_pick_order[counter]
    if len(stock_pick_order) == 1:
        stock_pick_order = []

/var/folders/rx/s42f27kn5j5ddqzlhz7qr2yw0000gn/T/ipykernel_14533/4200785160.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rack_All['Ext Orderno'][rack_row_no] = orderno
/var/folders/rx/s42f27kn5j5ddqzlhz7qr2yw0000gn/T/ipykernel_14533/4200785160.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rack_All['totalQty'][rack_row_no] += order['sku_qty'][order_row_no].get(skucode)
/var/folders/rx/s42f27kn5j5ddqzlhz7qr2yw0000gn/T/ipykernel_14533/1988780041.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/p

In [233]:
#len(find_order_no(get_skucode_from_stockno(69)))
cost = 0
for i in locus:
    cost += i//5

cost

41059